<a href="https://colab.research.google.com/github/zw2497/Twitter_Stream_Processing/blob/master/Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
consumer_key = "voHiOtlHtgaHC92ek5NYTABAZ"
consumer_secret = "qHra6hGcG7wWEEgTP0mrysqHquaLaloF2tkHcNJfM1tcskImgj"
access_token = "1114974821434241024-BZrPQPmONxVrb6NUVnVa6QQwBpy1An"
access_token_secret = "lRqIvcWURI351ixfT5QMogFgHNajKAwG2cTVPKeclGNXP"

In [0]:
# 暂时不需要Tweepy了
# import tweepy

# class MyStreamListener(tweepy.StreamListener):

#    def on_status(self, status):
#        print(status.text)

#    def on_error(self, status_code):
#        if status_code == 420:
#            #returning False in on_data disconnects the stream
#            return False

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tweepy.API(auth)

# myStreamListener = MyStreamListener()
# myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
# myStream.filter(languages=['english'])

In [2]:
!pip install pycurl
!pip install oauth2

In [0]:
import time
import pycurl
import json
import oauth2 as oauth

from sys import version_info
if version_info.major != 2:
  import urllib.parse as urllib
else:
  import urllib

In [0]:
API_ENDPOINT_URL = 'https://stream.twitter.com/1.1/statuses/filter.json'
USER_AGENT = 'TwitterStream 1.0' # This can be anything really

In [5]:
# You need to replace these with your own values
OAUTH_KEYS = {'consumer_key': consumer_key,
              'consumer_secret': consumer_secret,
              'access_token_key': access_token,
              'access_token_secret': access_token_secret}
print(OAUTH_KEYS)

{'consumer_key': 'voHiOtlHtgaHC92ek5NYTABAZ', 'consumer_secret': 'qHra6hGcG7wWEEgTP0mrysqHquaLaloF2tkHcNJfM1tcskImgj', 'access_token_key': '1114974821434241024-BZrPQPmONxVrb6NUVnVa6QQwBpy1An', 'access_token_secret': 'lRqIvcWURI351ixfT5QMogFgHNajKAwG2cTVPKeclGNXP'}


In [0]:
POST_PARAMS = {'include_entities': 0,
               'stall_warning': 'true',
               'track': 'iphone'}

In [0]:
class TwitterStream:
    def __init__(self, timeout=False):
        self.oauth_token = oauth.Token(key=OAUTH_KEYS['access_token_key'], secret=OAUTH_KEYS['access_token_secret'])
        self.oauth_consumer = oauth.Consumer(key=OAUTH_KEYS['consumer_key'], secret=OAUTH_KEYS['consumer_secret'])
        self.conn = None
        self.buffer = ''
        self.timeout = timeout
        self.setup_connection()

        
    def setup_connection(self):
        """ Create persistant HTTP connection to Streaming API endpoint using cURL.
        """
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        # Restart connection if less than 1 byte/s is received during "timeout" seconds
        if isinstance(self.timeout, int):
            self.conn.setopt(pycurl.LOW_SPEED_LIMIT, 1)
            self.conn.setopt(pycurl.LOW_SPEED_TIME, self.timeout)
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL)
        self.conn.setopt(pycurl.USERAGENT, USER_AGENT)
        # Using gzip is optional but saves us bandwidth.
        self.conn.setopt(pycurl.ENCODING, 'deflate, gzip')
        self.conn.setopt(pycurl.POST, 1)
        self.conn.setopt(pycurl.POSTFIELDS, urllib.urlencode(POST_PARAMS))
        self.conn.setopt(pycurl.HTTPHEADER, ['Host: stream.twitter.com',
                                             'Authorization: %s' % self.get_oauth_header()])
        # self.handle_tweet is the method that are called when new tweets arrive
        self.conn.setopt(pycurl.WRITEFUNCTION, self.handle_tweet)

        
    def get_oauth_header(self):
        """ Create and return OAuth header.
        """
        params = {'oauth_version': '1.0',
                  'oauth_nonce': oauth.generate_nonce(),
                  'oauth_timestamp': str(int(time.time()))}
       
        req = oauth.Request(method='POST', parameters=params, url='%s?%s' % (API_ENDPOINT_URL,
                                                                             urllib.urlencode(POST_PARAMS)))
        req.sign_request(oauth.SignatureMethod_HMAC_SHA1(), self.oauth_consumer, self.oauth_token)
        
        req = str(req.to_header()['Authorization']).encode('utf-8')
        return req.decode('utf-8')

      
    def start(self):
        """ Start listening to Streaming endpoint.
        Handle exceptions according to Twitter's recommendations.
        """
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                # Network error, use linear back off up to 16 seconds
                print('Network error: %s' % self.conn.errstr())
                print('Waiting %s seconds before trying again' % backoff_network_error)
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)
            if sc == 420:
                # Rate limit, use exponential back off starting with 1 minute and double each attempt
                print('Rate limit, waiting %s seconds' % backoff_rate_limit)
                time.sleep(backoff_rate_limit)
                backoff_rate_limit *= 2
            else:
                # HTTP error, use exponential back off up to 320 seconds
                print('HTTP error %s, %s' % (sc, self.conn.errstr()))
                print('Waiting %s seconds' % backoff_http_error)
                time.sleep(backoff_http_error)
                backoff_http_error = min(backoff_http_error * 2, 320)

                
    def handle_tweet(self, data):
        """ This method is called when data is received through Streaming endpoint.
        """
        self.buffer += data.decode('utf-8')
        
        if data.endswith(b'\r\n') and self.buffer.strip():
            # complete message received
            message = json.loads(self.buffer)
            self.buffer = ''
            msg = ''
            if message.get('limit'):
                print('Rate limiting caused us to miss %s tweets' % (message['limit'].get('track')))
            elif message.get('disconnect'):
                raise Exception('Got disconnect: %s' % message['disconnect'].get('reason'))
            elif message.get('warning'):
                print('Got warning: %s' % message['warning'].get('message'))
            else:
                print(message)
                print('Got tweet with text: %s' % message.get('text'))

In [35]:
ts = TwitterStream()
ts.setup_connection()
ts.start()

{"created_at":"Thu Apr 18 04:01:39 +0000 2019","id":1118726260018962433,"id_str":"1118726260018962433","text":"RT @zani0: iPhone\u306e\u30ab\u30e1\u30e9\u306b\u30ec\u30fc\u30b6\u30fc\u304c\u76f4\u6483\u3057\u3066\u3076\u3063\u58ca\u308c\u308b\u77ac\u9593\u64ae\u308c\u3066\u305f\u3002\n\u307f\u3093\u306a\u3082\u30af\u30e9\u30d6\u3067iPhone\u4f7f\u3046\u6642\u306f\u30ec\u30fc\u30b6\u30fc\u306b\u6c17\u3092\u3064\u3051\u3088\u3046\u306d\uff01 https:\/\/t.co\/NE042oeP8a","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":903242140737708032,"id_str":"903242140737708032","name":"Dr.DJ\u52dd\u5229","screen_name":"DrumnDJMetal","location":"\u65e5\u672c","url":null,"description":"Drum&DJ&\u30c7\u30b9\u30dc\u30a4\u30b9 EDM\u3068Rock\u304c\u597d\u3

KeyboardInterrupt: ignored

Network error: Failed writing body (0 != 6417)
Waiting 0.25 seconds before trying again
{"created_at":"Thu Apr 18 04:01:44 +0000 2019","id":1118726280759971841,"id_str":"1118726280759971841","text":"@luccassshenriqu Iphone tem umas paradinha esquisita","display_text_range":[17,52],"source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":1118726181644316672,"in_reply_to_status_id_str":"1118726181644316672","in_reply_to_user_id":844208674956611584,"in_reply_to_user_id_str":"844208674956611584","in_reply_to_screen_name":"luccassshenriqu","user":{"id":849002754060234753,"id_str":"849002754060234753","name":"R\u0394\u03a5SS\u0394 M\u0394T\u03a6S","screen_name":"rayssamatosss","location":"BELORY HILLS","url":"https:\/\/www.instagram.com\/rayssamatosss","description":"\u2022 \ud83d\udcb8 \uff36\uff49\uff56\uff45\uff4e\uff44\uff4f \uff41\uff56\uff41\uff4e\u00e7\uff41\uff44\uff4f \ud83d\u

KeyboardInterrupt: ignored

Network error: Failed writing body (0 != 3192)
Waiting 1.25 seconds before trying again


KeyboardInterrupt: ignored